In [1]:
import os
os.getcwd()

'/home/baki/atsi/UNEM_Gaussian'

## Usage of different methods for fewshot classifications tasks

In [1]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from src.methods.U_Paddle import U_Paddle
from src.methods.U_Paddle_embt import U_Paddle_EMBT
from src.utils import compute_confidence_interval, load_checkpoint, Logger, extract_mean_features, extract_features
from src.methods.tim import ALPHA_TIM, TIM_GD
from src.methods.paddle import PADDLE
from src.methods.soft_km import SOFT_KM
from src.methods.laplacianshot import LaplacianShot
from src.methods.paddle_gd import PADDLE_GD
from src.methods.ici import ICI
from src.methods.bdcspn import BDCSPN
from src.methods.baseline import Baseline
from src.methods.pt_map import PT_MAP
from src.datasets import Tasks_Generator, get_dataset, get_dataloader, SamplerSupport, SamplerQuery, CategoriesSampler
import os
from src.utils import load_pickle, save_pickle
import random
import matplotlib.pyplot as plt
from src.datasets.loader import DatasetFolder
from src.datasets.transform import with_augment, without_augment
from torch.utils.data import DataLoader
from src.models.ResNet import *
from torch.optim.lr_scheduler import StepLR
from src.unroll_trainer import *

from sklearn.metrics import accuracy_score


import pandas as pd

In [2]:
df1 = pd.read_pickle('../Bride_fshot/train_softmax_RN50_T30.plk')
df2 = pd.read_pickle('../Bride_fshot/train_visual_RN50.plk')

## Training the UTRAINER bone 

In [5]:
indices = torch.randperm(df1['concat_features'].size(0))

# Shuffle both features and labels using the same indices
df2 = {
    'concat_features': df2['concat_features'][indices],
    'concat_labels': df2['concat_labels'][indices]
}

In [6]:
all_features_support = df2['concat_features'][:250]
all_labels_support = df2['concat_labels'][:250].long()
all_features_query = df2['concat_features'][250:]
all_labels_query = df2['concat_labels'][250:].long()

In [70]:
class taelik():
    s_shots = [2,4,8,16,20]
    k_eff = 2
    n_ways = 2
    n_query = 10
    queries = [2,4,6,8,10]
    device = 'cuda'
    sampling = 'dirichlet'
    n_batch = 1

In [ ]:
results = []
param_history = []
loss_histories = []
for n_query in taelik.queries:
    for shot in taelik.s_shots:
        sampler = CategoriesSampler(all_labels_support, all_labels_query, taelik.n_batch,
                                            taelik.k_eff, taelik.n_ways, shot, taelik.n_query, 
                                            taelik.sampling, 1, 0.1)
        sampler.create_list_classes(all_labels_support, all_labels_query)
        sampler_support = SamplerSupport(sampler)
        sampler_query = SamplerQuery(sampler)
        test_loader_query = []
        for indices in sampler_query :
            test_loader_query.append((all_features_query[indices,:], all_labels_query[indices]))
        
        test_loader_support = []
        for indices in sampler_support :
            test_loader_support.append((all_features_support[indices,:], all_labels_support[indices]))
            
        task_generator = Tasks_Generator(k_eff=taelik.k_eff, n_ways=taelik.n_ways, shot=shot, n_query=taelik.n_query, loader_support=test_loader_support, loader_query=test_loader_query, train_mean=torch.tensor([0]), log_file='./logs/mini/resnet18/uniform/PADDLE/PADDLE_run_0.log')
        u_paddle = U_Paddle(n_layers=10, device='cuda', log_file='xx.log')
        
        tasks = task_generator.generate_tasks()
        
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(u_paddle.parameters(), lr=0.1)
        scheduler = StepLR(optimizer, step_size=50, gamma=0.5)
        
        u_paddle, loss_history, params_history = training2(u_paddle, tasks, criterion, optimizer, scheduler, 50, 'cuda')
        torch.cuda.empty_cache()
        param_history.append(params_history)
        results.append(u_paddle)
        loss_histories.append(loss_history)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 378.31it/s]


Gradient of gamma: 8.900556713342667e-05
Epoch 0 - Loss: 0.6931601166725159
Parameter name: gamma
Parameter shape: torch.Size([])
Parameter value: Parameter containing:
tensor(0.9000, device='cuda:0', requires_grad=True)




  4%|█▊                                          | 2/50 [00:00<00:03, 14.20it/s]

Gradient of gamma: 4.247087053954601e-05
Epoch 1 - Loss: 0.6931537985801697
Parameter name: gamma
Parameter shape: torch.Size([])
Parameter value: Parameter containing:
tensor(0.8075, device='cuda:0', requires_grad=True)




100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 373.18it/s]


...


 20%|████████▌                                  | 10/50 [00:00<00:02, 15.54it/s]

Gradient of gamma: 4.259976776666008e-06
Epoch 9 - Loss: 0.6931521892547607
Parameter name: gamma
Parameter shape: torch.Size([])
Parameter value: Parameter containing:
tensor(0.2712, device='cuda:0', requires_grad=True)




100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 469.95it/s]


Gradient of gamma: 3.713004161909339e-06
Epoch 10 - Loss: 0.6931520700454712
Parameter name: gamma
Parameter shape: torch.Size([])
Parameter value: Parameter containing:
tensor(0.2261, device='cuda:0', requires_grad=True)




 24%|██████████▎                                | 12/50 [00:00<00:02, 16.17it/s]

In [ ]:
i = 3
x = results[i].run_task(task_dic=tasks, shot = taelik.s_shots[i])

In [ ]:
print(f"Confidence interval : {compute_confidence_interval(x['acc'][:, -1])[0]}")

In [ ]:
x['predictions'].cpu().numpy().argmax(2)

In [ ]:
tasks['y_q'].cpu().numpy()[0].T

In [ ]:
print(f"accuracy score : {accuracy_score(x['predictions'].cpu().numpy().argmax(2).T, tasks['y_q'].cpu().numpy()[0])}")

## Laplacian Shot

In [3]:
import torch.nn.functional as F
from tqdm import tqdm
import torch
import time
from numpy import linalg as LA
import numpy as np
import math
from scipy import sparse
import matplotlib
matplotlib.use('Agg')
from sklearn.neighbors import NearestNeighbors
from src.utils import get_metric, Logger, get_one_hot

#### Tools

In [154]:
def normalize(Y_in):
    maxcol = np.max(Y_in, axis=1)
    Y_in = Y_in - maxcol[:, np.newaxis]
    N = Y_in.shape[0]
    size_limit = 150000
    if N > size_limit:
        batch_size = 1280
        Y_out = []
        num_batch = int(math.ceil(1.0 * N / batch_size))
        for batch_idx in range(num_batch):
            start = batch_idx * batch_size
            end = min((batch_idx + 1) * batch_size, N)
            tmp = np.exp(Y_in[start:end, :])
            tmp = tmp / (np.sum(tmp, axis=1)[:, None])
            Y_out.append(tmp)
        del Y_in
        Y_out = np.vstack(Y_out)
    else:
        Y_out = np.exp(Y_in)
        Y_out = Y_out / (np.sum(Y_out, axis=1)[:, None])

    return Y_out

def normalization(z_s, z_q, train_mean, norm_type):
    """
        inputs:
            z_s : np.Array of shape [n_task, s_shot, feature_dim]
            z_q : np.Array of shape [n_task, q_shot, feature_dim]
            train_mean: np.Array of shape [feature_dim]
    """
    z_s = z_s.cpu()
    z_q = z_q.cpu()

    # CL2N Normalization
    if norm_type == 'CL2N':
        z_s = z_s - train_mean
        z_s = z_s / LA.norm(z_s, 2, 2)[:, :, None]
        z_q = z_q - train_mean
        z_q = z_q / LA.norm(z_q, 2, 2)[:, :, None]

    # L2 Normalization
    elif norm_type == 'L2N':
        z_s = z_s / LA.norm(z_s, 2, 2)[:, :, None]
        z_q = z_q / LA.norm(z_q, 2, 2)[:, :, None]
    return z_s, z_q


def run_prediction(support, query, y_s, y_q, shot, batch_size, lmd, bound_iter):
    """
    Corresponds to the LaplacianShot inference
    inputs:
        support : torch.Tensor of shape [n_task, s_shot, feature_dim]
        query : torch.Tensor of shape [n_task, q_shot, feature_dim]
        y_s : torch.Tensor of shape [n_task, s_shot]
        y_q : torch.Tensor of shape [n_task, q_shot]
    records :
        accuracy
        ent_energy
        inference time
    """
    pred = []
    for i in tqdm(range(batch_size)):
        
        substract = support[i][:, None, :] - query[i]
        distance = LA.norm(substract, 2, axis=-1)
        unary = distance.transpose() ** 2
        W = create_affinity(query[i], knn=5)
        preds, acc_list, ent_energy, times = bound_update(unary=unary, kernel=W, bound_lambda=lmd, y_s=y_s, y_q=y_q, task_i=i,
                                            bound_iteration=bound_iter)
        pred.append(preds)
    return pred, y_q, acc_list

def create_affinity(X,knn):
    N, D = X.shape

    nbrs = NearestNeighbors(n_neighbors=knn).fit(X)
    dist, knnind = nbrs.kneighbors(X)

    row = np.repeat(range(N), knn - 1)
    col = knnind[:, 1:].flatten()
    data = np.ones(X.shape[0] * (knn - 1))
    W = sparse.csc_matrix((data, (row, col)), shape=(N, N), dtype=float)
    return W

    
def bound_update(unary, kernel, bound_lambda, y_s, y_q, task_i, bound_iteration=20, batch=False):
    oldE = float('inf')
    Y = normalize(-unary)
    E_list = []
    out_list = []
    acc_list = []
    timestamps = []
    t0 = time.time()
    for i in range(bound_iteration):
        additive = -unary
        mul_kernel = kernel.dot(Y)
        Y = -bound_lambda * mul_kernel
        additive = additive - Y
        Y = normalize(additive)
        E = entropy_energy(Y, unary, kernel, bound_lambda, batch)
        E_list.append(E)
        # print('entropy_energy is ' +repr(E) + ' at iteration ',i)
        l = np.argmax(Y, axis=1)
        #out = np.take(y_s, l)
        out = l
        timestamps.append(time.time()-t0)

        if (i > 1 and (abs(E - oldE) <= 1e-6 * abs(oldE))):
            # print('Converged')
            out_list.append(torch.from_numpy(out))
            acc_list.append((torch.from_numpy(y_q[task_i]) == torch.from_numpy(out)).float())
            for j in range(bound_iteration-i-1):
                out_list.append(out_list[i].detach().clone())
                acc_list.append(acc_list[i].detach().clone())
                E_list.append(E_list[i])
                timestamps.append(0)
            break

        else:
            oldE = E.copy()

            out_list.append(torch.from_numpy(out))
            acc_list.append((torch.from_numpy(y_q[task_i]) == torch.from_numpy(out)).float())
        t0 = time.time()

    out_list = torch.stack(out_list, dim=0)
    acc_list = torch.stack(acc_list, dim=0).mean(dim=1, keepdim=True)

    return out, acc_list, E_list, timestamps
def entropy_energy(Y, unary, kernel, bound_lambda, batch=False):
    tot_size = Y.shape[0]
    pairwise = kernel.dot(Y)
    if batch == False:
        temp = (unary * Y) + (-bound_lambda * pairwise * Y)
        E = (Y * np.log(np.maximum(Y, 1e-20)) + temp).sum()
    else:
        batch_size = 1024
        num_batch = int(math.ceil(1.0 * tot_size / batch_size))
        E = 0
        for batch_idx in range(num_batch):
            start = batch_idx * batch_size
            end = min((batch_idx + 1) * batch_size, tot_size)
            temp = (unary[start:end] * Y[start:end]) + (-bound_lambda * pairwise[start:end] * Y[start:end])
            E = E + (Y[start:end] * np.log(np.maximum(Y[start:end], 1e-20)) + temp).sum()

    return E


def proto_rectification(y_s, support, query, shot,temp = 2, number_tasks=5):
    """
        inputs:
            support : np.Array of shape [n_task, s_shot, feature_dim]
            query : np.Array of shape [n_task, q_shot, feature_dim]
            shot: Shot

        ouput:
            proto_weights: prototype of each class
    """
    eta = support.mean(1) - query.mean(1)  # Shifting term
    query = query + eta[:, np.newaxis, :]  # Adding shifting term to each normalized query feature
    query_aug = torch.cat((support, query), axis=1)  # Augmented set S' (X')
    one_hot = get_one_hot(y_s)
    counts = one_hot.sum(1).view(support.size()[0], -1, 1)
    weights = one_hot.transpose(1, 2).matmul(support)
    init_prototypes = weights / counts

    proto_weights = []
    for j in tqdm(range(number_tasks)):
        distance = get_metric('cosine')(init_prototypes[j], query_aug[j])
        predict = torch.argmin(distance, dim=1)
        cos_sim = F.cosine_similarity(query_aug[j][:, None, :], init_prototypes[j][None, :, :], dim=2)  # Cosine similarity between X' and Pn
        cos_sim = temp * cos_sim
        W = F.softmax(cos_sim, dim=1)
        init_prototypeslist = [(W[predict == i, i].unsqueeze(1) * query_aug[j][predict == i]).mean(0, keepdim=True) for i in predict.unique()]
        proto = torch.cat(init_prototypeslist, dim=0).cpu()  # Rectified prototypes P'n
        
        if proto.shape[0] != len(torch.unique(y_s)):
            proto = init_prototypes[j]

        proto_weights.append(proto)

    proto_weights = np.stack(proto_weights, axis=0)
    return proto_weights

#### Task generating

In [125]:
sampler = CategoriesSampler(all_labels_support, all_labels_query, taelik.n_batch,
                                    taelik.k_eff, taelik.n_ways, 4, taelik.n_query, 
                                    taelik.sampling, 1, 0.1)
sampler.create_list_classes(all_labels_support, all_labels_query)
sampler_support = SamplerSupport(sampler)
sampler_query = SamplerQuery(sampler)
test_loader_query = []
for indices in sampler_query :
    test_loader_query.append((all_features_query[indices,:], all_labels_query[indices]))

test_loader_support = []
for indices in sampler_support :
    test_loader_support.append((all_features_support[indices,:], all_labels_support[indices]))
    
task_generator = Tasks_Generator(k_eff=taelik.k_eff, n_ways=taelik.n_ways, shot=5, n_query=taelik.n_query, loader_support=test_loader_support, loader_query=test_loader_query, train_mean=torch.tensor([0]), log_file='./logs/mini/resnet18/uniform/PADDLE/PADDLE_run_0.log')
u_paddle = U_Paddle(n_layers=10, device='cuda', log_file='xx.log')

task_dic = task_generator.generate_tasks()

#### Predictions

In [155]:
device = 'cuda'
proto_rect = True
y_s, y_q = task_dic['y_s'], task_dic['y_q']
x_s, x_q = task_dic['x_s'], task_dic['x_q']
train_mean = torch.tensor([0])

support, query = normalization(z_s=x_s, z_q=x_q, train_mean=train_mean, norm_type='L2N')
y_s = y_s.squeeze(2).to(device)
y_q = y_q.squeeze(2).to(device)
support = support.to(device)
query = query.to(device)

if proto_rect:
    support = proto_rectification(y_s=y_s, support=support, query=query, shot=shot)
else:
    one_hot = get_one_hot(y_s)
    counts = one_hot.sum(1).view(support.size()[0], -1, 1)
    weights = one_hot.transpose(1, 2).matmul(support)
    support = weights / counts
    support = support.cpu().numpy()
query = query.cpu().numpy()
y_s = y_s.cpu().numpy()
y_q = y_q.cpu().numpy()

# Run adaptation
pred, y_q, acc_list = run_prediction(support=support, query=query, y_s=y_s, y_q=y_q,batch_size=5,lmd =0.1,bound_iter=5, shot=6)



/tmp/ipykernel_15842/3793071448.py:43: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  z_s = z_s / LA.norm(z_s, 2, 2)[:, :, None]
/tmp/ipykernel_15842/3793071448.py:44: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  z_q = z_q / LA.norm(z_q, 2, 2)[:, :, None]
100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 11.17it/s]


In [156]:
print(f"Confidence interval : {compute_confidence_interval(acc_list)[0]}")

Confidence interval : [0.8]


# Paddle